# **ChatGPT Pandas Project**

### **🧹 Data Cleaning Assignment (Step by Step with Explanations)**

**You are given a dirty dataset (Customers.csv) with:**

    Missing values
    Duplicate rows
    Inconsistent text values
    Extra columns to calculate

In [199]:
import pandas as pd
import numpy as np

**Replace blanks with 0**

In [200]:
df = pd.read_csv("Customers.csv")

**1. Identify Missing Data**

**👉 Question:**

    Check which columns have missing values. Show both the count and the percentage of missing values for each column.

In [201]:
null_values = pd.DataFrame({
    "Columns": df.columns,
    "Null_count": df.isnull().sum().values,
    "Null_%": (df.isnull().sum() / len(df) * 100).map("{:.2f}%".format).values
})
null_values = null_values[null_values["Null_count"] > 0]
null_values

,Columns,Null_count,Null_%
3,Quantity_Sold,2,10.00%
4,Sales,1,5.00%
6,Customer_Rating,3,15.00%
8,Discount,3,15.00%


**2. Remove Duplicate Rows**

**👉 Question:**

    Find rows where Product_Name, Category, and Region are the same (exact duplicates).
    Remove those duplicates, but keep the first occurrence.

*(This step ensures that the dataset doesn’t count the same product twice.)*

In [202]:
df[["Product_Name", "Category", "Region", "Supplier"]].value_counts()

Product_Name     Category     Region      Supplier       
Laptop           Electronics  Dhaka       TechWorld          3
Microwave        Appliances   Sylhet      KitchenPro         2
Air Conditioner  Appliances   Dhaka       HomeApplianceCo    2
Tablet           Electronics  Khulna      MegaTech           2
Camera           Electronics  Barisal     TechWorld          1
Headphones       Electronics  Rajshahi    SoundMax           1
Blender          Appliances   Chittagong  KitchenPro         1
Projector        Electronics  Dhaka       VisionTech         1
Refrigerator     Appliances   Dhaka       HomeApplianceCo    1
Smartwatch       Electronics  Rajshahi    MegaTech           1
Smartphone       Electronics  Chittagong  MegaTech           1
Speaker          Electronics  Chittagong  SoundMax           1
Television       Electronics  Dhaka       VisionTech         1
Vacuum Cleaner   Appliances   Khulna      HomeApplianceCo    1
Washing Machine  Appliances   Sylhet      HomeApplianceCo   

In [203]:
len(df)

20

In [204]:
duplicated = df[["Product_Name", "Category", "Region", "Supplier"]].duplicated()
len(df[~duplicated]) # 20
df = df[~duplicated]
len(df)

15

In [205]:
df["Product_Name"].value_counts()

Product_Name
Laptop             1
Smartphone         1
Refrigerator       1
Microwave          1
Headphones         1
Television         1
Blender            1
Air Conditioner    1
Tablet             1
Camera             1
Smartwatch         1
Vacuum Cleaner     1
Speaker            1
Washing Machine    1
Projector          1
Name: count, dtype: int64

**3. Handle Missing Values**

**👉 Question:**

Fix the missing values by applying the following rules:

    If Sales is missing → replace it with the average Sales of the entire column.
    If Discount is missing → replace it with 0 (means no discount was given).
    If Customer_Rating is missing → replace it with the median rating of the dataset.
    If Quantity_Sold is missing → replace it with the mean quantity sold of the dataset.

*(This step makes sure your dataset is complete and doesn’t have blanks anymore.)*

In [206]:
df.head(1)

,Product_ID,Product_Name,Category,Quantity_Sold,Sales,Region,Customer_Rating,Supplier,Discount,Returned_Units
0,101,Laptop,Electronics,15.0,15000.0,Dhaka,4.5,TechWorld,5.0,1


In [ ]:
# If Sales is missing → replace it with the average Sales of the entire column.
df["Sales"].fillna(df["Sales"].mean().__round__(2), inplace=True)

In [ ]:
# If Discount is missing → replace it with 0 (means no discount was given).
df["Discount"].fillna(0, inplace=True)

In [ ]:
# If Customer_Rating is missing → replace it with the median rating of the dataset.
Median = df["Customer_Rating"].sort_values().agg("median") # Median ---> middle value --> 4.2
df["Customer_Rating"].fillna(Median, inplace=True)

In [ ]:
# If Quantity_Sold is missing → replace it with the mean quantity sold of the dataset.
# df["Quantity_Sold"].isnull().sum()
df["Quantity_Sold"].fillna(df["Quantity_Sold"].mean().__round__(2), inplace=True)

**4. Fix Data Consistency**

**👉 Question:**

    Sometimes text values are written differently but mean the same thing.
    Make sure Region values are consistent (e.g., "dhaka", "Dhaka ", "Dhaka" → all become "Dhaka").
    Make sure Category only contains "Electronics" or "Appliances". If there are typos or variations (like "Electronic", "Appliance"), fix them.

*(This step prevents confusion during analysis, because "Dhaka" and "dhaka" should not be counted separately.)*

In [211]:
df.head(1)

,Product_ID,Product_Name,Category,Quantity_Sold,Sales,Region,Customer_Rating,Supplier,Discount,Returned_Units
0,101,Laptop,Electronics,15.0,15000.0,Dhaka,4.5,TechWorld,5.0,1


In [212]:
df[(df["Category"] == "Electronic") | (df["Category"] == "Appliance")] # Found 0

,Product_ID,Product_Name,Category,Quantity_Sold,Sales,Region,Customer_Rating,Supplier,Discount,Returned_Units


In [213]:
df[(df["Region"] == "Dhaka") | (df["Region"] == "dhaka")] # All is right

,Product_ID,Product_Name,Category,Quantity_Sold,Sales,Region,Customer_Rating,Supplier,Discount,Returned_Units
0,101,Laptop,Electronics,15.00,15000.0,Dhaka,4.5,TechWorld,5.0,1
2,103,Refrigerator,Appliances,10.00,12000.0,Dhaka,4.0,HomeApplianceCo,7.0,1
5,106,Television,Electronics,12.00,11400.0,Dhaka,4.1,VisionTech,8.0,1
7,108,Air Conditioner,Appliances,16.29,25000.0,Dhaka,4.3,HomeApplianceCo,15.0,1
18,119,Projector,Electronics,6.00,12000.0,Dhaka,4.3,VisionTech,0.0,0


**5. Create New Columns (Feature Engineering)**

**👉 Question:**

    Create two new calculated columns:
    Revenue_After_Discount = Sales - (Sales * Discount / 100)

*(This shows the actual revenue after applying the discount.)*

    Return_Rate = (Returned_Units / Quantity_Sold) * 100

*(This shows what percentage of sold products were returned.)*

In [214]:
df.head(1)

,Product_ID,Product_Name,Category,Quantity_Sold,Sales,Region,Customer_Rating,Supplier,Discount,Returned_Units
0,101,Laptop,Electronics,15.0,15000.0,Dhaka,4.5,TechWorld,5.0,1


**Create two new calculated columns:**

**Column-1:**

    Revenue_After_Discount = Sales - (Sales * Discount / 100)

In [215]:
df["Revenue_After_Discount"] = df["Sales"] - (df["Sales"] * df["Discount"] / 100)
df.head(3)

,Product_ID,Product_Name,Category,Quantity_Sold,Sales,Region,Customer_Rating,Supplier,Discount,Returned_Units,Revenue_After_Discount
0,101,Laptop,Electronics,15.0,15000.0,Dhaka,4.5,TechWorld,5.0,1,14250.0
1,102,Smartphone,Electronics,25.0,20000.0,Chittagong,4.2,MegaTech,10.0,0,18000.0
2,103,Refrigerator,Appliances,10.0,12000.0,Dhaka,4.0,HomeApplianceCo,7.0,1,11160.0


**Column-2:**

    Return_Rate = (Returned_Units / Quantity_Sold) * 100

In [216]:
## step-1: make this "Returned_Units" <-- obj/str to float using ".astype(float)"
df["Returned_Units"] = df["Returned_Units"].astype(float)

## step-2: then apply this
Return_Rate = ((df["Returned_Units"] / df["Quantity_Sold"]) * 100).__round__(2)
df["Return_Rate"] = Return_Rate

In [217]:
df.head(4)

,Product_ID,Product_Name,Category,Quantity_Sold,Sales,Region,Customer_Rating,Supplier,Discount,Returned_Units,Revenue_After_Discount,Return_Rate
0,101,Laptop,Electronics,15.0,15000.0,Dhaka,4.5,TechWorld,5.0,1.0,14250.0,6.67
1,102,Smartphone,Electronics,25.0,20000.0,Chittagong,4.2,MegaTech,10.0,0.0,18000.0,0.00
2,103,Refrigerator,Appliances,10.0,12000.0,Dhaka,4.0,HomeApplianceCo,7.0,1.0,11160.0,10.00
3,104,Microwave,Appliances,8.0,4000.0,Sylhet,3.8,KitchenPro,0.0,0.0,4000.0,0.00


**6. Save the Clean Dataset**

**👉 Question:**

    After cleaning the dataset:
    There should be no missing values.
    There should be no duplicate rows.
    Text values should be consistent.

*Finally, save the cleaned dataset into a new CSV file called:*
*sales_data_clean.csv*

*(This file will be your final, fully usable dataset.)*

**Creating New Data-Set CSV File**

In [218]:
df.to_csv("New_DataSet.csv", index=False)

**Your dataset fully meets all the requirements we set:** *<-- Chacked by ChatGPT*

1. Nulls handled ✅
2. Duplicates removed ✅
3. Text standardized ✅
4. Revenue and return rate calculated ✅